In [22]:
import os
from typing import List, Tuple

import cv2

import numpy as np

from keras.applications.inception_v3 import InceptionV3
from keras import models, layers, losses
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from tensorflow.python.keras.models import load_model
from sklearn.model_selection import train_test_split

BASE_DATASET_PATH = 'dataset'
IMAGE_WIDTH = 299
IMAGE_HEIGHT = 299
EXAMPLES_QUANTITY = 5400
CATEGORIES_QUANTITY = 10
TEST_DISTRIBUTION = 0.2

In [23]:
def get_labels_list() -> List[str]:
    """Get list of labels from txt file."""
    try:
        with open(os.path.join(BASE_DATASET_PATH, 'name of the animals.txt')) as labels_file:
            labels = labels_file.read().splitlines()
    except FileNotFoundError:
        raise FileNotFoundError('Incorrect path to "name of the animals.txt" file!')

    return labels

In [24]:
def image_to_array(image_path: str) -> np.array:
    """
    Transform image to numpy array with required standards.

    :param image_path: Image path
    :return: Image as numpy array
    """
    return cv2.resize(cv2.imread(image_path), (IMAGE_WIDTH, IMAGE_HEIGHT)) / 255.

In [25]:
labels = get_labels_list()

full_dataset_x = np.zeros((EXAMPLES_QUANTITY, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
full_dataset_y = np.zeros(EXAMPLES_QUANTITY)
samples_counter = 0

for label_i, label in enumerate(labels):
    label_folder = os.path.join(BASE_DATASET_PATH, 'animals', label)

    if not os.path.exists(label_folder):
        raise ValueError(f'Dataset doesn\'t have {label}\'s folder!')

    for filename in os.listdir(label_folder):
        example_path = os.path.join(label_folder, filename)

        if os.path.isfile(example_path):
            image = image_to_array(example_path)

            full_dataset_x[samples_counter] = image
            full_dataset_y[samples_counter] = label_i

            samples_counter += 1

In [26]:
train_x, test_x, train_y, test_y = train_test_split(full_dataset_x, full_dataset_y,
                                                    test_size=TEST_DISTRIBUTION,
                                                    random_state=42, shuffle=True)

train_y = to_categorical(train_y, num_classes=CATEGORIES_QUANTITY)
test_y = to_categorical(test_y, num_classes=CATEGORIES_QUANTITY)

In [27]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])

model.compile(
    optimizer='adam',
    loss=losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [28]:
model.fit(train_x, train_y, epochs=20, validation_split=0.3, batch_size=32, shuffle=True)

MemoryError: Unable to allocate 6.04 GiB for an array with shape (3024, 299, 299, 3) and data type float64

In [ ]:
model.evaluate(self.test_x, self.test_y, batch_size=32)

## Predict

In [ ]:
image_array = image_to_array('butterfly.jpg')
image_array = image_array.reshape(1, image_array.shape[0], image_array.shape[1], 3)
predict = model.predict(image_array)
idx = np.argmax(predict)
print(predict)
print(idx)
print(_get_label(idx))